In [23]:
#Requirements for the spark workflow
from sedona.spark import *
from pyspark.sql.functions import col, count, countDistinct
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType, DateType
from sedona.spark.sql.st_constructors import ST_Point
from sedona.spark.sql.st_functions import GeometryType, ST_XMin, ST_YMin, ST_XMax, ST_YMax
from sedona.spark import SedonaKepler
from pyspark.sql import functions as F
from sedona.spark.geopandas import GeoDataFrame, read_parquet
from sedona.sql import st_predicates

import sys, os
from shapely.geometry import Point
from itertools import product
import sedona
from sedona.spark import SedonaContext

import sedona.db
import numpy as np
import leafmap



In [2]:
# For anonymous access to public S3 buckets
#sd_cont is needed to read all the csv in
sd_cont = (
    SedonaContext.builder()
    .config(
        "spark.hadoop.fs.s3a.bucket.bucket-name.aws.credentials.provider",
        "org.apache.hadoop.fs.s3a.AnonymousAWSCredentialsProvider",
    )
    .getOrCreate()
)


sd = SedonaContext.create(sd_cont)

25/11/07 20:37:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.
25/11/07 20:37:32 WARN UDTRegistration: Cannot register UDT for org.geotools.coverage.grid.GridCoverage2D, which is already registered.
25/11/07 20:37:32 WARN SimpleFunctionRegistry: The function rs_union_aggr replaced a previously registered function.
25/11/07 20:37:32 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.geom.Geometry, which is already registered.
25/11/07 20:37:32 WARN UDTRegistration: Cannot register UDT for org.apache.sedona.common.S2Geography.Geography, which is already registered.
25/11/07 20:37:32 WARN UDTRegistration: Cannot register UDT for org.locationtech.jts.index.SpatialIndex, which is already registered.
25/11/07 20:37:32 WARN SimpleFunctionRegistry: The function st_envelope_aggr replaced a previously registered function.
25/11/07 20:37:32 WARN SimpleFunctionRegistry: The function st_intersection_aggr replaced a previously 

In [3]:

# Path to the directory containing your CSV files
directory_path = "../2024/"

df = sd_cont.read.option("header", True).format("csv").load(directory_path)


In [4]:
#Setting the ST_Point after getting unique values
df.select(ST_Point(col("LONGITUDE"), col("LATITUDE")).alias("GEOMETRY"), "STATION", "DATE", "LATITUDE", "LONGITUDE", "ELEVATION", "NAME", "REPORT_TYPE", 
          "SOURCE", "HourlyDryBulbTemperature", "HourlyPressureChange", "HourlyPressureTendency", "HourlySeaLevelPressure", "HourlyStationPressure", 
          "HourlyWindDirection", "HourlyWindGustSpeed", "HourlyWindSpeed").filter(col("HourlyPressureChange").isNotNull()).write.format("geoparquet").mode("overwrite").save("./p_diff")
df_p_diff_pq = sd.read.format("geoparquet").load("./p_diff")


AttributeError: module 'sedona' has no attribute 'read'

In [ ]:
#I'm not going to use sedonaDB because the df is 8.7gb (from 70gb in text files)
#and I only have 12 gb memmory allocted to docker
df_p_diff_pq.explain('cost')

In [7]:
#This has to be build with unique locations, or at least stations
df_p_diff_pq.select("GEOMETRY").distinct().write.format("geoparquet").mode("overwrite").save("./stations")
df_stations_pq = sd.read.format("geoparquet").load("./stations")
df_stations_pq.show(5)
df_stations_pq.count()

+--------------------+
|            GEOMETRY|
+--------------------+
|  POINT (14.35 49.2)|
|POINT (136.9 37.3...|
|POINT (-60.983333...|
|   POINT (1.4 44.75)|
|POINT (139.366666...|
+--------------------+
only showing top 5 rows



25/11/07 20:44:07 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr


9156

In [9]:
OVERTURE_RELEASE = "2025-09-24.0"
COUNTRY_CODES_OF_INTEREST = ["US"]
SOURCE_DATA_URL = f"s3a://overturemaps-us-west-2/release/{OVERTURE_RELEASE}/theme=divisions/type=division_area"
OUTPUT_FILE = "my_super_cool_data.parquet"

country_overlap_condition = F.arrays_overlap(
    F.col("country"),
    F.array(*[F.lit(x.upper()) for x in COUNTRY_CODES_OF_INTEREST]),
)

source_df = (
    sd.read.format("geoparquet")
    .load(SOURCE_DATA_URL)
    .filter(col("country").isin(COUNTRY_CODES_OF_INTEREST))
    #.filter(col("region")=='US-CA')
    .filter(col("subtype")=='country')
    .withColumn("_overture_release_version", F.lit(OVERTURE_RELEASE))
    .withColumn("_ingest_timestamp", F.current_timestamp())
)

source_df.selectExpr("geometry", "country").write.format("geoparquet").mode("overwrite").save("./USA_geom")
USA_geom_pq = sd.read.format("geoparquet").load("./USA_geom")
USA_geom_pq.show(5)
map = SedonaKepler.create_map(USA_geom_pq, name="USA")
map

+--------------------+-------+
|            geometry|country|
+--------------------+-------+
|MULTIPOLYGON (((-...|     US|
|MULTIPOLYGON (((-...|     US|
|MULTIPOLYGON (((-...|     US|
|MULTIPOLYGON (((-...|     US|
+--------------------+-------+



[Stage 16:=====================================================>  (25 + 1) / 26]

KeplerGl(data={'USA': {'index': [0, 1, 2, 3], 'columns': ['geometry', 'country'], 'data': [['MULTIPOLYGON (((-…

25/11/07 20:45:54 ERROR FileFormatWriter: Aborting job ba48bf7d-d73a-434f-a1a9-15091e458d07.
java.io.FileNotFoundException: File file:/opt/workspace/ra/workspace/USA_geom/_temporary/0 does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.listStatus(RawLocalFileSystem.java:597)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.fs.ChecksumFileSystem.listStatus(ChecksumFileSystem.java:761)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:1972)
	at org.apache.hadoop.fs.FileSystem.listStatus(FileSystem.java:2014)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.getAllCommittedTaskPaths(FileOutputCommitter.java:334)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.commitJobInternal(FileOutputCommitter.java:404)
	at org.apache.hadoop.mapreduce.lib.output.FileOutputCommitter.commitJob(FileOutputCommitter.java:377)
	at org.apache.parquet.ha

In [11]:
# 2. Define a coordinate grid with a 0.5-degree step
longitude_step = .5
latitude_step = .5

longitudes = [i * longitude_step for i in range(int(-180 / longitude_step), int(180 / longitude_step) + 1)]
latitudes = [i * latitude_step for i in range(int(-90 / latitude_step), int(90 / latitude_step) + 1)]

# 3. Generate a list of all coordinate pairs
coordinate_pairs = list(product(longitudes, latitudes))

# 4. Create a Spark DataFrame from the list of coordinates
schema = ["longitude", "latitude"]
df_lat_lon = sd.createDataFrame(coordinate_pairs, schema=schema)

# 5. Create the Sedona geometry points
# ST_Point takes longitude first, then latitude.
spatial_df = df_lat_lon.withColumn(
    "interp_geom",
    F.expr(f"ST_Point(longitude, latitude)")
)
# Show the resulting DataFrame clipped to USA boundaries
print("Generated spatial DataFrame:")
df_usa_points = spatial_df.join(
    USA_geom_pq,
    st_predicates.ST_Within(spatial_df["interp_geom"], USA_geom_pq["geometry"]),
    "inner"
)
# Show the results
df_usa_points.show(5)
df_usa_points.printSchema()

Generated spatial DataFrame:


+---------+--------+-------------------+--------------------+-------+
|longitude|latitude|        interp_geom|            geometry|country|
+---------+--------+-------------------+--------------------+-------+
|   -179.0|    51.5|  POINT (-179 51.5)|MULTIPOLYGON (((-...|     US|
|   -179.0|    51.5|  POINT (-179 51.5)|MULTIPOLYGON (((-...|     US|
|   -179.0|    52.0|    POINT (-179 52)|MULTIPOLYGON (((-...|     US|
|   -179.0|    52.0|    POINT (-179 52)|MULTIPOLYGON (((-...|     US|
|   -178.5|    28.5|POINT (-178.5 28.5)|MULTIPOLYGON (((-...|     US|
+---------+--------+-------------------+--------------------+-------+
only showing top 5 rows

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- interp_geom: geometry (nullable = true)
 |-- geometry: geometry (nullable = true)
 |-- country: string (nullable = true)



AnalysisException: [PATH_NOT_FOUND] Path does not exist: file:/opt/workspace/ra/workspace/.interp_points.

In [13]:
%%time
df_usa_points.drop(*["longitude", "latitude", "geometry", "country"]).write.format("geoparquet").mode("overwrite").save("./interp_points")
df_interp_pq = sd.read.format("geoparquet").load("./interp_points")
df_interp_pq.show(5)
df_interp_pq.printSchema()
print(df_interp_pq.count())
print(spatial_df.count())

25/11/07 20:51:47 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr
25/11/07 20:51:47 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr
25/11/07 20:51:47 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr
25/11/07 20:51:47 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr
25/11/07 20:51:47 WARN GeoParquetFileFormat: GeoParquet currently does not support vectorized reader. Falling back to parquet-mr


+-----------------+
|      interp_geom|
+-----------------+
|  POINT (-108 32)|
|  POINT (-108 32)|
|  POINT (-108 32)|
|  POINT (-108 32)|
|POINT (-108 32.5)|
+-----------------+
only showing top 5 rows

root
 |-- interp_geom: geometry (nullable = true)

18636
260281
CPU times: user 29.2 ms, sys: 6.89 ms, total: 36.1 ms
Wall time: 17.4 s


In [14]:
df_stations_pq.createOrReplaceTempView("stations")
df_interp_pq.createOrReplaceTempView("interp")

In [16]:
%%time
#Adapted From https://wherobots.com/blog/introducing-knn-join-for-wherobots-and-apache-sedona/
df_knn_join = sd.sql("""
SELECT
    stations.GEOMETRY AS stations_GEOM,
    interp.interp_GEOM,
    ST_DISTANCESPHERE(stations.GEOMETRY, interp.interp_GEOM) AS DISTANCE
FROM 
    stations
JOIN 
    interp 
ON 
    ST_KNN(stations.GEOMETRY, interp.interp_GEOM, 10, FALSE)
""")
df_knn_join.show(5)
df_knn_join.write.format("geoparquet").mode("overwrite").save("./knn_join")
df_knn_pq = sd.read.format("geoparquet").load("./knn_join")


+------------------+------------------+-----------------+
|     stations_GEOM|       interp_GEOM|         DISTANCE|
+------------------+------------------+-----------------+
|POINT (14.35 49.2)|POINT (-67.5 45.5)|5869061.575762225|
|POINT (14.35 49.2)|POINT (-67.5 45.5)|5869061.575762225|
|POINT (14.35 49.2)|POINT (-67.5 45.5)|5869061.575762225|
|POINT (14.35 49.2)|POINT (-67.5 45.5)|5869061.575762225|
|POINT (14.35 49.2)|  POINT (-67.5 45)|5901613.263144317|
+------------------+------------------+-----------------+
only showing top 5 rows

CPU times: user 6.75 ms, sys: 4.69 ms, total: 11.4 ms
Wall time: 937 ms


In [17]:
df_p_diff_pq.createOrReplaceTempView("pressure_diff")
df_knn_pq.createOrReplaceTempView("stations_knn")

In [18]:
%%time
df_iwd_grid = sd.sql("""
SELECT 
    stations_knn.interp_GEOM,
    SUM(ABS(pressure_diff.HourlyPressureChange) / POWER(stations_knn.DISTANCE, 2)) / SUM(1 / POWER(stations_knn.DISTANCE, 2)) AS interpolated_value,
    AVG(stations_knn.DISTANCE) as avg_dist
    from 
      stations_knn
    inner join 
      pressure_diff 
    on 
      stations_knn.stations_GEOM = pressure_diff.GEOMETRY
    group by stations_knn.interp_geom
""")
df_iwd_grid.write.format("geoparquet").mode("overwrite").save("./knn_join")
df_iwd_pq = sd.read.format("geoparquet").load("./knn_join")
df_iwd_pq.show(5)


+------------------+-------------------+------------------+
|       interp_GEOM| interpolated_value|          avg_dist|
+------------------+-------------------+------------------+
| POINT (-112 33.5)|0.03225291529728004|18833.823674307456|
|POINT (-75.5 44.5)|0.03262977226930969|185723.04136612354|
|    POINT (-89 48)|0.03342035015265212| 583809.3943930377|
|  POINT (-78 43.5)|0.03257088761273111|  169648.553352226|
| POINT (-111 39.5)| 0.0248577929465244|24208.072334796652|
+------------------+-------------------+------------------+
only showing top 5 rows



+------------------+-------------------+------------------+
|       interp_GEOM| interpolated_value|          avg_dist|
+------------------+-------------------+------------------+
| POINT (-112 33.5)|0.03225291529728004|18833.823674307456|
|POINT (-75.5 44.5)|0.03262977226930969|185723.04136612354|
|    POINT (-89 48)|0.03342035015265212| 583809.3943930377|
|  POINT (-78 43.5)|0.03257088761273111|  169648.553352226|
| POINT (-111 39.5)| 0.0248577929465244|24208.072334796652|
+------------------+-------------------+------------------+
only showing top 5 rows

CPU times: user 49.5 ms, sys: 10.7 ms, total: 60.2 ms
Wall time: 32 s


In [32]:
df_iwd_pq.createOrReplaceTempView("iwd")


In [39]:
df_min_max = sd.sql("""
SELECT
    Min(ST_XMin(interp_GEOM)) as x_min,
    Max(ST_XMax(interp_GEOM)) as x_max,
    Min(ST_YMin(interp_GEOM)) as y_min,
    Max(ST_YMax(interp_GEOM)) as y_max
from iwd
WHERE ST_XMax(interp_GEOM) < 0

""")
df_min_max.show()


+------+-----+-----+-----+
| x_min|x_max|y_min|y_max|
+------+-----+-----+-----+
|-178.5|-67.5| 19.0| 71.5|
+------+-----+-----+-----+



In [47]:
# Define parameters  This should be .25 deg off of the points so there is one point per cell
offset_x = df_min_max.first()['x_min'] -0.25
offset_y = df_min_max.first()['y_max'] +0.25
cell_size = 0.5
width = df_min_max.first()['x_max'] - df_min_max.first()['x_min'] + 1
height = df_min_max.first()['y_max'] - df_min_max.first()['y_min'] + 1
srid = 4326 # Example SRID (WGS84)
num_bands = 1 # Number of bands
pixel_type = 'D' # Pixel type, e.g., 'D' for Double

# Create an empty raster using SQL expression
raster_df = sd.sql(f"""
    SELECT RS_MakeEmptyRaster({num_bands}, '{pixel_type}', {width}, {height}, {offset_x}, {offset_y}, {cell_size}) AS raster
""")

# Show schema to verify
raster_df.printSchema()
raster_df.show(1, False)

root
 |-- raster: raster (nullable = true)

+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|raster                                                                                                                                                                                                                                                                                                                                               |
+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
#Player around.  Use some of this code.
df_pressure_diff = df.select(ST_Point(col("LONGITUDE"), col("LATITUDE")),"STATION", "DATE", "LATITUDE", "LONGITUDE", "ELEVATION", "NAME", "REPORT_TYPE", "SOURCE", "HourlyDryBulbTemperature", "HourlyPressureChange", "HourlyPressureTendency", "HourlySeaLevelPressure", "HourlyStationPressure", "HourlyWindDirection", "HourlyWindGustSpeed", "HourlyWindSpeed").filter(col("HourlyPressureChange").isNotNull()).show()
df.filter(col("HourlySeaLevelPressure").isNotNull() | col("HourlyStationPressure").isNotNull()).count()
#counts of records with different filters
df.count()
#130,112,717 37,352,572 77,319,947 77,210,243
aggregated = df.groupby('STATION', 'DATE').agg({abs('HourlyPressureChange'): 'min', abs('HourlyPressureChange'): 'min'})
df['AbsPressure'] = df['HourlyPressureChange'].abs()
from pyspark.sql import functions as F
df = df.withColumn("absPressure", F.abs(F.col("HourlyPressureChange")))
df.tail(10)
df.agg(F.min('HourlyPressureChange')).show()
df.filter(col('HourlyPressureChange').isNotNull()).withColumn("HourlyPressureChange", col("HourlyPressureChange").cast(FloatType())).groupby('STATION', 'DATE')\
.agg({abs('HourlyPressureChange'): 'min', abs('HourlyPressureChange'): 'min'})

In [ ]:
#Using Overtures instread
#import geopandas as gpd

#url = "https://naciscdn.org/naturalearth/110m/cultural/ne_110m_admin_0_countries.zip"


#gdf = gpd.read_file(url)
#gdf
#df_conus = sedona.createDataFrame(gdf[(gdf.SOV_A3=='US1') & (gdf.TYPE=='Country')][['SOVEREIGNT', 'geometry']])
#map = SedonaKepler.create_map(df=df_conus, name="CONUS")
#map


In [ ]:
spatial_df.first()['geometry']

In [ ]:
#Used for testing with just CA
#CA_geom = source_df.selectExpr("geometry", "region")
#CA_geom.show(5)
#map1 = SedonaKepler.create_map(CA_geom, name="CA")
#map1
#
#CA_geom = source_df.selectExpr("geometry", "region").filter(GeometryType(col('geometry'))=='MULTIPOLYGON')
#CA_geom.show(5)
#map2 = SedonaKepler.create_map(CA_geom, name="CA")
#map2
#
#CA_geom = source_df.selectExpr("geometry", "region").filter(GeometryType(col('geometry'))=='POLYGON')
#CA_geom.show(5)
#map3 = SedonaKepler.create_map(CA_geom, name="CA")
#map3